## Make a montage for unbiased cell classification


This is separate from the segmentation process. This is useful for visualizing the segmented images in a single view. This combined image/montage will be used to create a labelled image for further processing and cell classification.

I assume that you already have binarized images (for e.g., generated by `cell_segmentation.ipynb`) in place. 

## Setup the environment and load the path of the images

In [49]:
%reset -f

In [50]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import tifffile
from matplotlib.path import Path

matplotlib.use('TKAgg')
plt.ion()
plt.rc("axes", grid=False)
path_of_separate_files = "/home/tatsatb/Data/MASKS/" # Path to the folder containing separate mask files


## Load the tiff files

In [51]:
# Find all tiff files in the specified directory
tiff_files = glob.glob(os.path.join(path_of_separate_files, "*.tif")) + glob.glob(
    os.path.join(path_of_separate_files, "*.tiff"))

# Dictionary to store all the loaded arrays
loaded_arrays = {}

## Make the montage

- We will combine the images into a single image with two rows
- We will add padding around each cell
- We will save the combined image

In [52]:

# Load each tiff file into a numpy array named after the file
for file_path in tiff_files:
    # Extract filename without extension and use as variable name
    filename = os.path.basename(file_path)
    array_name = os.path.splitext(filename)[0]

    # Load the TIFF file
    print(f"Loading {filename}...")
    loaded_arrays[array_name] = tifffile.imread(file_path).transpose(1, 2, 0).astype(np.int64) / 255

    # Create a variable in the global namespace with the array name
    globals()[array_name] = loaded_arrays[array_name]

    # Print information about the loaded array
    print(f"  Loaded {array_name}: shape {loaded_arrays[array_name].shape}, dtype {loaded_arrays[array_name].dtype}")

print(f"Loaded {len(loaded_arrays)} TIFF files from {path_of_separate_files}")

# Create a two-row combined image with padding around each cell
# Find max height, width and frames
all_arrays = list(loaded_arrays.values())
array_names = list(loaded_arrays.keys())
n_arrays = len(all_arrays)
padding = 100  # Padding around each cell in pixels
arrays_per_row = (n_arrays + 1) // 2  # Distribute arrays evenly across two rows

# Add padding to each array
padded_arrays = []
for i, arr in enumerate(all_arrays):
    height, width, frames = arr.shape
    padded_height = height + 2 * padding
    padded_width = width + 2 * padding
    padded_arr = np.zeros((padded_height, padded_width, frames), dtype=np.int64)
    padded_arr[padding:padding + height, padding:padding + width, :frames] = arr
    padded_arrays.append(padded_arr)
    print(f"Added padding to {array_names[i]}: {arr.shape} → {padded_arr.shape}")

# Calculate dimensions for each row
row_heights = []
row_widths = []
max_frames = max(arr.shape[2] for arr in padded_arrays)

# First row
first_row_arrays = padded_arrays[:arrays_per_row]
row_heights.append(max(arr.shape[0] for arr in first_row_arrays))
row_widths.append(sum(arr.shape[1] for arr in first_row_arrays))

# Second row
second_row_arrays = padded_arrays[arrays_per_row:]
if second_row_arrays:
    row_heights.append(max(arr.shape[0] for arr in second_row_arrays))
    row_widths.append(sum(arr.shape[1] for arr in second_row_arrays))
else:
    row_heights.append(0)
    row_widths.append(0)

# Create the combined array
total_height = sum(row_heights)
max_width = max(row_widths)
combined_image = np.zeros((total_height, max_width, max_frames), dtype=np.int64)

# Add arrays to the first row
current_x = 0
current_y = 0
for i in range(arrays_per_row):
    arr = padded_arrays[i]
    height, width, frames = arr.shape
    combined_image[current_y:current_y + height, current_x:current_x + width, :frames] = arr
    current_x += width

# Add arrays to the second row
current_x = 0
current_y = row_heights[0]
for i in range(arrays_per_row, n_arrays):
    arr = padded_arrays[i]
    height, width, frames = arr.shape
    combined_image[current_y:current_y + height, current_x:current_x + width, :frames] = arr
    current_x += width

# # Save the combined image
combined_path = os.path.join(path_of_separate_files, 'Combined', 'combined_mask_2rows.tif')

os.makedirs(os.path.dirname(combined_path), exist_ok=True)

tifffile.imwrite(combined_path, (combined_image * 255).astype(np.uint8).transpose(2, 0, 1))
print(f"Combined image shape: {combined_image.shape}")


## Visualize the combined image

In [53]:
plt.figure(figsize=(8, 8))
ax1 = plt.gca()
plt.imshow(combined_image[:, :, 0], cmap='viridis')
ax1.grid(False)
plt.colorbar(shrink=0.5)
plt.title('A sample frame from the combined image')
plt.tight_layout()
plt.show()